In [16]:
# =========================================
# Week 2 - Data Cleaning (E-Commerce Project)
# Dataset: Customer_support_data.csv
# =========================================

import pandas as pd
import numpy as np
from tabulate import tabulate
from IPython.display import display

# ---- 1. Load Dataset ----
df_before = pd.read_csv("Customer_support_data.csv")   # Keep original for comparison
df = df_before.copy()  # Working copy

print("📌 Dataset Loaded Successfully!")

# ---- 2. Dataset Shape ----
print("\n📌 Dataset Shape (Before Cleaning):")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

# ---- 3. Missing Values Count ----
print("\n📌 Missing Values Per Column (Before Cleaning):")
missing_values = df.isnull().sum().reset_index()
missing_values.columns = ["Column", "Missing Values"]
print(tabulate(missing_values, headers="keys", tablefmt="grid"))

# ---- 4. Handle Missing Values ----
missing_mask = df.isnull()   # Store NaN locations before filling

filling_strategy = []
for col in df.columns:
    if df[col].dtype in ["int64","float64"]:  # Numeric column
        mean_val = round(df[col].mean(skipna=True),2) if not df[col].dropna().empty else np.nan
        median_val = round(df[col].median(skipna=True),2) if not df[col].dropna().empty else np.nan
        mode_val = df[col].mode().iloc[0] if not df[col].mode().empty else np.nan
        strategy = "Median"
        df[col] = df[col].fillna(median_val)
    else:  # Categorical column
        mean_val, median_val = np.nan, np.nan
        mode_val = df[col].mode().iloc[0] if not df[col].mode().empty else "N/A"
        strategy = "Mode"
        df[col] = df[col].fillna(mode_val)
    
    filling_strategy.append([col, mean_val, median_val, mode_val, strategy])

# Print Filling Strategy Table
print("\n📌 Missing Value Filling Strategy (Mean / Median / Mode):")
print(tabulate(pd.DataFrame(filling_strategy, 
                            columns=["Column","Mean","Median","Mode","Used Strategy"]),
               headers="keys", tablefmt="grid", showindex=False))

# ---- 5. Remove Duplicates ----
before_dup = df.shape[0]
df = df.drop_duplicates()
after_dup = df.shape[0]
print(f"\n📌 Duplicate Rows Removed: {before_dup - after_dup}")







# ---- 6. Outlier Detection & Treatment with Explanation ----
print("\n📌 Quartile & Outlier Explanation (Numeric Columns):")

iqr_summary = []  # store results for summary table

for col in df.select_dtypes(include=["int64","float64"]).columns:
    Q1 = df[col].quantile(0.25)
    Q2 = df[col].quantile(0.50)  # Median
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    below_outliers = (df[col] < lower).sum()
    above_outliers = (df[col] > upper).sum()
    
    # Print explanation with conceptual meaning
    print(f"\n📊 Column: {col}")
    print(f"   Q1 (middle of lower half) = {Q1:.2f}")
    print(f"   Q2 (median / middle of dataset) = {Q2:.2f}")
    print(f"   Q3 (middle of upper half) = {Q3:.2f}")
    print(f"   IQR (Q3 - Q1) = {IQR:.2f}")
    print(f"   Lower Bound = Q1 - 1.5*IQR = {lower:.2f}")
    print(f"   Upper Bound = Q3 + 1.5*IQR = {upper:.2f}")
    
    # Added note explaining bounds
    print(f"   Note: Values >= {lower:.2f} and <= {upper:.2f} are considered normal;")
    print(f"         only values < {lower:.2f} or > {upper:.2f} are outliers.")
    
    print(f"   ➝ Outliers Found: {below_outliers} below, {above_outliers} above")
    
    # Store results for table
    iqr_summary.append([col, round(Q1,2), round(Q2,2), round(Q3,2), round(IQR,2),
                        round(lower,2), round(upper,2), below_outliers, above_outliers])
    
    # Cap the outliers
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])

# Create summary table
iqr_table = pd.DataFrame(iqr_summary, 
                         columns=["Column", "Q1", "Q2 (Median)", "Q3", "IQR", 
                                  "Lower Bound", "Upper Bound", "Outliers Below", "Outliers Above"])

print("\n📌 IQR & Outlier Summary Table:")
print(tabulate(iqr_table, headers="keys", tablefmt="grid", showindex=False))

# ---- 7. Show First 20 Rows Before & After Cleaning ----
print("\n📌 First 20 Rows BEFORE Cleaning (Original Data with NaNs):")
display(df_before.head(20))

print("\n📌 First 20 Rows AFTER Cleaning (Missing Values Filled, Highlighted in Yellow):")

def highlight_filled(val, was_missing):
    return "background-color: #D97D55" if was_missing else ""

styled = df.head(20).style.apply(
    lambda s: [highlight_filled(v, was_missing) 
               for v, was_missing in zip(s, missing_mask[s.name].head(20))],
    axis=0
)
display(styled)

# ---- 8. Summary After Cleaning ----
print("\n📌 Dataset Shape (After Cleaning):")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n📌 Missing Values Per Column (After Cleaning):")
missing_values_after = df.isnull().sum().reset_index()
missing_values_after.columns = ["Column", "Missing Values"]
print(tabulate(missing_values_after, headers="keys", tablefmt="grid"))






📌 Dataset Loaded Successfully!

📌 Dataset Shape (Before Cleaning):
Rows: 85907, Columns: 20

📌 Missing Values Per Column (Before Cleaning):
+----+-------------------------+------------------+
|    | Column                  |   Missing Values |
+====+=========================+==================+
|  0 | Unique id               |                0 |
+----+-------------------------+------------------+
|  1 | channel_name            |                0 |
+----+-------------------------+------------------+
|  2 | category                |                0 |
+----+-------------------------+------------------+
|  3 | Sub-category            |                0 |
+----+-------------------------+------------------+
|  4 | Customer Remarks        |            57165 |
+----+-------------------------+------------------+
|  5 | Order_id                |            18232 |
+----+-------------------------+------------------+
|  6 | order_date_time         |            68693 |
+----+----------------------

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5
5,1cfde5b9-6112-44fc-8f3b-892196137a62,Email,Returns,Fraudulent User,NaN,a2938961-2833-45f1-83d6-678d9555c603,NaN,01/08/2023 15:13,01/08/2023 18:39,01-Aug-23,NaN,NaN,NaN,NaN,Desiree Newton,Emma Park,John Smith,0-30,Morning,5
6,11a3ffd8-1d6b-4806-b198-c60b5934c9bc,Outcall,Product Queries,Product Specific Information,NaN,bfcb562b-9a2f-4cca-aa79-fd4e2952f901,NaN,01/08/2023 15:31,01/08/2023 23:52,01-Aug-23,NaN,NaN,NaN,NaN,Shannon Hicks,Aiden Patel,Olivia Tan,>90,Morning,5
7,372b51a5-fa19-4a31-a4b8-a21de117d75e,Inbound,Returns,Exchange / Replacement,Very good,88537e0b-5ffa-43f9-bbe2-fe57a0f4e4ae,NaN,01/08/2023 16:17,01/08/2023 16:23,01-Aug-23,NaN,NaN,NaN,NaN,Laura Smith,Evelyn Kimura,Jennifer Nguyen,On Job Training,Evening,5
8,6e4413db-4e16-42fc-ac92-2f402e3df03c,Inbound,Returns,Missing,Shopzilla app and it's all coustomer care serv...,e6be9713-13c3-493c-8a91-2137cbbfa7e6,NaN,01/08/2023 21:03,01/08/2023 21:07,01-Aug-23,NaN,NaN,NaN,NaN,David Smith,Nathan Patel,John Smith,>90,Split,5
9,b0a65350-64a5-4603-8b9a-a24a4a145d08,Inbound,Shopzilla Related,General Enquiry,NaN,c7caa804-2525-499e-b202-4c781cb68974,NaN,01/08/2023 23:31,01/08/2023 23:36,01-Aug-23,NaN,NaN,NaN,NaN,Tabitha Ayala,Amelia Tanaka,Michael Lee,31-60,Evening,5



📌 First 20 Rows AFTER Cleaning (Missing Values Filled, Highlighted in Yellow):


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,Good,c27c9bb4-fa36-4140-9f1f-21009254ffdb,09/08/2023 11:55,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5.000000
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,Good,d406b0c7-ce17-4654-b9de-f08d421254bd,09/08/2023 11:55,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5.000000
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,Good,c273368d-b961-44cb-beaf-62d6fd6c00d5,09/08/2023 11:55,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5.000000
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,Good,5aed0059-55a4-4ec6-bb54-97942092020a,09/08/2023 11:55,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5.000000
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,Good,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,09/08/2023 11:55,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5.000000
5,1cfde5b9-6112-44fc-8f3b-892196137a62,Email,Returns,Fraudulent User,Good,a2938961-2833-45f1-83d6-678d9555c603,09/08/2023 11:55,01/08/2023 15:13,01/08/2023 18:39,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Desiree Newton,Emma Park,John Smith,0-30,Morning,5.000000
6,11a3ffd8-1d6b-4806-b198-c60b5934c9bc,Outcall,Product Queries,Product Specific Information,Good,bfcb562b-9a2f-4cca-aa79-fd4e2952f901,09/08/2023 11:55,01/08/2023 15:31,01/08/2023 23:52,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Shannon Hicks,Aiden Patel,Olivia Tan,>90,Morning,5.000000
7,372b51a5-fa19-4a31-a4b8-a21de117d75e,Inbound,Returns,Exchange / Replacement,Very good,88537e0b-5ffa-43f9-bbe2-fe57a0f4e4ae,09/08/2023 11:55,01/08/2023 16:17,01/08/2023 16:23,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Laura Smith,Evelyn Kimura,Jennifer Nguyen,On Job Training,Evening,5.000000
8,6e4413db-4e16-42fc-ac92-2f402e3df03c,Inbound,Returns,Missing,Shopzilla app and it's all coustomer care services is very good service provided all time,e6be9713-13c3-493c-8a91-2137cbbfa7e6,09/08/2023 11:55,01/08/2023 21:03,01/08/2023 21:07,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,David Smith,Nathan Patel,John Smith,>90,Split,5.000000
9,b0a65350-64a5-4603-8b9a-a24a4a145d08,Inbound,Shopzilla Related,General Enquiry,Good,c7caa804-2525-499e-b202-4c781cb68974,09/08/2023 11:55,01/08/2023 23:31,01/08/2023 23:36,01-Aug-23,HYDERABAD,Electronics,979.000000,427.000000,Tabitha Ayala,Amelia Tanaka,Michael Lee,31-60,Evening,5.000000



📌 Dataset Shape (After Cleaning):
Rows: 85907, Columns: 20

📌 Missing Values Per Column (After Cleaning):
+----+-------------------------+------------------+
|    | Column                  |   Missing Values |
+====+=========================+==================+
|  0 | Unique id               |                0 |
+----+-------------------------+------------------+
|  1 | channel_name            |                0 |
+----+-------------------------+------------------+
|  2 | category                |                0 |
+----+-------------------------+------------------+
|  3 | Sub-category            |                0 |
+----+-------------------------+------------------+
|  4 | Customer Remarks        |                0 |
+----+-------------------------+------------------+
|  5 | Order_id                |                0 |
+----+-------------------------+------------------+
|  6 | order_date_time         |                0 |
+----+-------------------------+------------------+
|  7 | Is